<a href="https://colab.research.google.com/github/dennisbarchetti/natural_language_processsing/blob/master/EsameLinguistica2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installazione delle librerie

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 14.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


Importo le librerie

In [ ]:
import nltk

import spacy #Modulo per elaborazione testo naturale (nlp)
from collections import Counter #modulo per contare (verbi/pronomi/nomi...)
import pandas as pd #database ricco di dati (già filtrati/elaborati)
import numpy as np;
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

from nltk.corpus import wordnet

Download del dizionario:

In [ ]:
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/chat80.zip.
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2000.zip.
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2002.zip.
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/dependency_treebank.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    

True

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
import os
os.listdir("/content/drive/MyDrive/Colab_Notebooks")
infile = open("/content/drive/MyDrive/Colab_Notebooks/Esame.txt")
sample_text = infile.read()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Ricerco tutti gli aggettivi presenti nel testo e li inserisco su un dataframe ordinati per numero di apparizioni:

In [ ]:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
# Process whole documents
text = ("""We shall be Conservative, in all cases where we think Conservatism essential to the public good;—and we shall be Radical in everything which may seem to us to require radical treatment and radical reform. 
          We do not believe that everything in Society is either exactly right or exactly wrong;—what is good we desire to preserve and improve;—what is evil, to exterminate, or reform.""")
  
#doc = nlp(text) #decommenta se come testo vuoi usare quello che c'è scritto sopra ed è associato alla variabile text
doc = nlp(sample_text)  #decommenta se come testo vuoi usare quello che c'è scritto sul documento Testo.txt salvato google drive 
  
# Token and Tag
for token in doc: 
  #print(token, token.pos_)

 aggettivi = []
for token in doc:
    if token.pos_ == "ADJ":
        #print(token, token.pos_)
        aggettivi.append(token.text) 
print(aggettivi)

numero_aggettivi= Counter(aggettivi)
dfA = pd.DataFrame(numero_aggettivi.most_common(),columns=['aggettivi', 'count']) 
dfA[:100] #primi 100 elementi del dataframe


['early', 'possible', 'unfriendly', 'rouble', 'possible', 'early', 'first', 'potential', 'final', 'German', 'precautionary', 'German', 'early', 'technological', 'prolonged', 'rouble', 'Russian', 'natural', 'lower', 'Russian', 'European', 'other', 'such']


,aggettivi,count
0,early,3
1,possible,2
2,rouble,2
3,German,2
4,Russian,2
5,unfriendly,1
6,first,1
7,potential,1
8,final,1
9,precautionary,1


inserisco in una lista i 3 aggettivi ripetuti più volte nel testo

In [ ]:
tre_ADJ_mostcommon = []
tre_ADJ_mostcommon = (numero_aggettivi.most_common(3))

print(numero_aggettivi.most_common(3)) #[('radical', 2), ('Conservative', 1), ('essential', 1)]   
#tre_ADJ_mostcommon è una lista coplessa a me interessa solo la parola e non il numero di volte che appare


listaADJ = [] # lista di parole filtrate da altre informazioni che non mi servono
i=0
while i< len(tre_ADJ_mostcommon):                                                         
  listaADJ.append(tre_ADJ_mostcommon[i][0]) #mi interessa la posizione 0 della coppia di (aggettivo,numero) dell'elemento i-esimo  
  i=i+1
print(listaADJ)

[('early', 3), ('possible', 2), ('rouble', 2)]
['early', 'possible', 'rouble']


ricerco i sinonimi per ognuno dei 3 aggettivi, stampo una lista di sinonimi per ognono di essi con anche il numero di elementi di ognuna 

Successivamente li inserisco in 3 liste attraverso le quali vado a creaare dei dataframe, i quali poi vengono uniti.

In [ ]:
list1 = []
list2 = []
list2 = []

i = 0
for aggettivo in listaADJ:
  tokens = nltk.word_tokenize(aggettivo)
  synonms = set()
  for token in tokens:
    synsets= wordnet.synsets(token) #estraggo insieme di sinonimi da database wordnet
    for syn in synsets: #per ogni sinonimo nell'insieme 
      for lemma in syn.lemmas():   #La lemmatizzazione è il processo di raggruppamento delle diverse forme flesse di una parola in modo che possano essere analizzate come un unico elemento.                                  
        synonms.add(lemma.name())   #lemma.name -->conservative                 #lemma.name -->conservativist  
        #print(syn.lemmas())        #[Lemma('conservative.n.01.conservative'), Lemma('conservative.n.01.conservativist')]
  numero_sinonimi = 0
  for sinonimo in synonms:
    numero_sinonimi= numero_sinonimi+1
  #print("sinonimi di " + aggettivo)
  #print(synonms)
  print(numero_sinonimi)
  if i == 0:
    list1 = synonms
  if i == 1:
    list2 = synonms 
  if i == 2:
    list3 = synonms 
  i = i + 1

print(list1)
print(list2)
print(list3)

i = 0
for aggettivo in listaADJ:
  if i == 0:
     df1 = pd.DataFrame(list1,columns=[aggettivo]) 
  if i == 1:
    df2 = pd.DataFrame(list2,columns=[aggettivo])
  if i == 2:
    df3 = pd.DataFrame(list3,columns=[aggettivo]) 
  i = i + 1

dfu = df1.combine_first(df2) #ho usato questa funzione per unire i tre dataframe i uno unico con 3 colonne
dfu = dfu.combine_first(df3)
dfu[:100] #primi 100 elementi del dataframe

# altre possibilità che però non funzionano
# df1.append(df2)
# df1.merge(df2, left_on=list1, right_on=list2)
# df1.join(...)


7
2
2
{'other', 'ahead_of_time', 'early_on', 'former', 'betimes', 'early', 'too_soon'}
{'possible', 'potential'}
{'rouble', 'ruble'}


,early,possible,rouble
0,other,possible,rouble
1,ahead_of_time,potential,ruble
2,early_on,NaN,NaN
3,former,NaN,NaN
4,betimes,NaN,NaN
5,early,NaN,NaN
6,too_soon,NaN,NaN


In [ ]:
df1[:100] #primi 100 elementi del dataframe

,early
0,other
1,ahead_of_time
2,early_on
3,former
4,betimes
5,early
6,too_soon


In [ ]:
df2[:100] #primi 100 elementi del dataframe

,possible
0,possible
1,potential


In [ ]:
df3[:100] #primi 100 elementi del dataframe

,rouble
0,rouble
1,ruble
